In [1]:
#load the variables specific to this dataset 
#on brahma, SNPs have been split by DL CV split 
snp_prefix="/srv/scratch/annashch/gecco/mpra_validation/pygl_rs72685323/splits"
model_prefix="/srv/scratch/annashch/deeplearning/gecco/crossvalid/v4/gecco.classification.SummitWithin200bpCenter"
test_set_prefix="/srv/scratch/annashch/deeplearning/gecco/crossvalid/v4/predictions"
n_folds=10 
num_tasks=5 
all_snps_basename="rs72685323_14_51359100_100k.collapsed.txt"
target_layer_idx=-2
tasks=['DNASEC','DNASEV','SW480','HCT116','COLO205']
outf_name="SNP_effect_predictions.selfcalibration.txt"
ref_fasta="/mnt/data/annotations/by_release/hg19.GRCh37/hg19.genome.fa"

In [2]:
#load the dependencies
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import pickle 
from abstention.calibration import PlattScaling, IsotonicRegression
from keras.models import load_model, Model 

from kerasAC.metrics import recall, specificity, fpr, fnr, precision, f1
from kerasAC.custom_losses import get_ambig_binary_crossentropy
from kerasAC.generators import * 
from kerasAC.predict import get_model_layer_functor, get_layer_outputs


Using TensorFlow backend.


## Get Ref/Alt predictions and pre-activations 

In [3]:
def load_keras_model(fold,model_prefix,target_layer_idx):    
    #load the model
    custom_objects={"recall":recall,
                    "sensitivity":recall,
                    "specificity":specificity,
                    "fpr":fpr,
                    "fnr":fnr,
                    "precision":precision,
                    "f1":f1,
                    "ambig_binary_crossentropy":get_ambig_binary_crossentropy()}
    model=load_model(".".join([model_prefix,str(fold)]),custom_objects=custom_objects)
    print("loaded model")
    #load the model to predict preacts 
    preact_model=Model(inputs=model.input,
                       outputs=model.layers[target_layer_idx].output)
    print("loaded preact model")
    return model,preact_model

In [4]:
def get_test_set_logits_and_labels(fold,test_set_prefix):
    test_set_pickle_name='.'.join([test_set_prefix,str(fold)])
    test_set_pickle=pickle.load(open(test_set_pickle_name,'rb'))
    test_set_labels=test_set_pickle[1]
    test_set_logits=test_set_pickle[2] 
    return test_set_labels, test_set_logits 

    

In [5]:
def get_calibration_function_classification(logits,labels):
    #need to generate separate calibration function for each task 
    num_tasks=logits.shape[1] 
    calibration_funcs=[] 
    for task_index in range(num_tasks):
        calibration_funcs.append(PlattScaling()(
            valid_preacts=logits[:,task_index],
            valid_labels=labels.iloc[:,task_index].astype(bool)))
    return calibration_funcs

In [6]:
def get_calibration_function_regression(preacts,labels):
    #need to generate separate calibration function for each task 
    num_tasks=preacts.shape[1]
    calibration_funcs=[] 
    for task_index in range(num_tasks):
        calibration_funcs.append(IsotonicRegression()(
            valid_preacts=preacts[:,task_index],
            valid_labels=labels.iloc[:,task_index]))
    return calibration_funcs

In [7]:
def get_snp_generators(fold,snp_prefix,all_snps_basename,ref_fasta):
    snp_file='/'.join([snp_prefix,all_snps_basename+"."+str(fold)])
    snps=pd.read_csv(snp_file,header=0,sep='\t')
    if snps.shape[0]==0: 
        return None,None
    snp_ref_generator=SNPGenerator(snp_file,
                                   ref_fasta=ref_fasta,
                                   allele_col="Ref")
    snp_alt_generator=SNPGenerator(snp_file,
                                   ref_fasta=ref_fasta,
                                   allele_col="Alt")
    return snps, snp_ref_generator,snp_alt_generator

In [8]:
def get_calibrated_predictions(logits,calibration_functions): 
    num_tasks=logits.shape[1] 
    num_calibration_functions=len(calibration_functions)
    assert num_tasks == num_calibration_functions 
    calibrated_predictions=None 
    for i in range(num_tasks): 
        task_calibrated_predictions=np.expand_dims(calibration_functions[i](logits[:,i]),axis=1)
        if calibrated_predictions is None: 
            calibrated_predictions=task_calibrated_predictions
        else: 
            calibrated_predictions=np.concatenate((calibrated_predictions,task_calibrated_predictions),axis=1)
    print(calibrated_predictions.shape)
    return calibrated_predictions 

In [9]:
#goal of first pass is to get the ref logits & SNP labels for calibration 
labels_all=None 
ref_preacts_all=None

for fold in range(n_folds): 
    print(fold)
    try:
        snps, snp_ref_generator,snp_alt_generator=get_snp_generators(fold,snp_prefix,all_snps_basename,ref_fasta)
        print("Got snp ref and alt generators")
    except:
        print("fold:"+str(fold)+" appears to be empty, skipping")
        continue 
    
    labels=snps.iloc[:,6::]
    if labels_all is None: 
        labels_all=labels
    else: 
        labels_all=np.concatenate((labels_all,labels),axis=0)
    model,preact_model = load_keras_model(fold,model_prefix,target_layer_idx)
    ref_preacts=preact_model.predict_generator(snp_ref_generator,
                max_queue_size=5000, 
                workers=0, 
                use_multiprocessing=False, 
                verbose=1)
    
    print("Got ref preacts")
    if ref_preacts_all is None: 
        ref_preacts_all=ref_preacts 
    else: 
        ref_preacts_all=np.concatenate((ref_preacts_all,ref_preacts),axis=0)

print("getting calibration function")
print(labels_all.shape)
print(ref_preacts_all.shape)

#need to generate separate calibration function for each task 
num_tasks=ref_preacts.shape[1] 
calibration_functions=[] 
for task_index in range(num_tasks):
    calibration_functions.append(PlattScaling()(
        valid_preacts=ref_preacts_all[:,task_index],
        valid_labels=labels_all.iloc[:,task_index].astype(bool)))

0
fold:0 appears to be empty, skipping
1
fold:1 appears to be empty, skipping
2
fold:2 appears to be empty, skipping
3
fold:3 appears to be empty, skipping
4
fold:4 appears to be empty, skipping
5
fold:5 appears to be empty, skipping
6
loaded labels
filtered on chroms_to_use
data.shape:(703, 8)
loaded labels
filtered on chroms_to_use
data.shape:(703, 8)
Got snp ref and alt generators


ResourceExhaustedError: OOM when allocating tensor with shape[200] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node batch_normalization_2/beta/Assign (defined at /users/annashch/miniconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:402)  = Assign[T=DT_FLOAT, _grappler_relax_allocator_constraints=true, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](batch_normalization_2/beta, batch_normalization_2/Const_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'batch_normalization_2/beta/Assign', defined at:
  File "/users/annashch/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/users/annashch/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/users/annashch/mini

In [ ]:
#Create the output file 
outf=open(outf_name,'w')
#Write the header 
outf.write("\t".join(['Chrom','StartPos','EndPos','Ref','Alt','Rsid']))
outf.write('\t'+'\t'.join(['Label.'+task for task in tasks]))
for field in ['RefLogit','RefCalibrated','AltLogit','AltCalibrated']: 
    for task in tasks: 
        outf.write('\t'+field+'.'+task)
outf.write('\n')

In [ ]:

for fold in range(n_folds): 
    print(fold)
    try:
        snps, snp_ref_generator,snp_alt_generator=get_snp_generators(fold,snp_prefix,all_snps_basename,ref_fasta)
        print("Got snp ref and alt generators")
    except:
        print("fold:"+str(fold)+" appears to be empty, skipping")
        continue 
    labels=snps.iloc[:,6::]
    
    model,preact_model = load_keras_model(fold,model_prefix,target_layer_idx)
    ref_preacts=preact_model.predict_generator(snp_ref_generator,
                max_queue_size=5000, 
                workers=0, 
                use_multiprocessing=False, 
                verbose=1)
    print("Got ref preacts")
    alt_preacts=preact_model.predict_generator(snp_alt_generator,
                max_queue_size=5000, 
                workers=0, 
                use_multiprocessing=False, 
                verbose=1)    
    print("Got alt preacts")    
    ref_calibrated_predictions=get_calibrated_predictions(ref_preacts,
                                                          calibration_functions)
    print("Got calibrated reference predictions")
    alt_calibrated_predictions=get_calibrated_predictions(alt_preacts,
                                                          calibration_functions)
    print("Got calibrated alternate predictions")
    
    num_snps=ref_calibrated_predictions.shape[0]
    for snp_index in range(num_snps): 
        outf.write('\t'.join([str(j) for j in snps.iloc[snp_index]])+
                   '\t'+
                   '\t'.join([str(j) for j in ref_preacts[snp_index]])+
                   '\t'+
                   '\t'.join([str(j) for j in ref_calibrated_predictions[snp_index]])+
                   '\t'+
                   '\t'.join([str(j) for j in alt_preacts[snp_index]])+
                   '\t'+
                   '\t'.join([str(j) for j in alt_calibrated_predictions[snp_index]])+
                   '\n')
outf.close() 

## ISM Plots 

In [ ]:
import pandas as pd 
snp_predictions=pd.read_csv(outf_name,header=0,sep='\t')
snp_predictions['Label.DNASEC']=snp_predictions['Label.DNASEC'].astype('category')
snp_predictions['Label.DNASEV']=snp_predictions['Label.DNASEV'].astype('category')
snp_predictions['Label.SW480']=snp_predictions['Label.SW480'].astype('category')
snp_predictions['Label.HCT116']=snp_predictions['Label.HCT116'].astype('category')
snp_predictions['Label.COLO205']=snp_predictions['Label.COLO205'].astype('category')


In [ ]:
import plotnine
from plotnine import * 

In [ ]:
snp_predictions['Target']=snp_predictions['Rsid']=='14_51359099_51359100_C_G'

### DNAse C 

In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefLogit.DNASEC",y="AltLogit.DNASEC",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Logit")+
ylab("Alternate Allele Logit")+
ggtitle("Healthy Controls"))


In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefCalibrated.DNASEC",y="AltCalibrated.DNASEC",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Calibrated Predictions")+
ylab("Alternate Allele Calibrated Predictions")+
ggtitle("Healthy Controls"))


## DNAse V 

In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefLogit.DNASEV",y="AltLogit.DNASEV",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Logit")+
ylab("Alternate Allele Logit")+
ggtitle("Tumor Samples"))


In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefCalibrated.DNASEV",y="AltCalibrated.DNASEV",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Calibrated Predictions")+
ylab("Alternate Allele Calibrated Predictions")+
ggtitle("Tumor Samples"))


## SW480

In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefLogit.SW480",y="AltLogit.SW480",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Logit")+
ylab("Alternate Allele Logit")+
ggtitle("SW480"))

In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefCalibrated.SW480",y="AltCalibrated.SW480",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Calibrated Predictions")+
ylab("Alternate Allele Calibrated Predictions")+
ggtitle("SW480"))


## HCT116

In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefLogit.HCT116",y="AltLogit.HCT116",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Logit")+
ylab("Alternate Allele Logit")+
ggtitle("HCT116"))

In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefCalibrated.HCT116",y="AltCalibrated.HCT116",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Calibrated Predictions")+
ylab("Alternate Allele Calibrated Predictions")+
ggtitle("HCT116"))


## COLO205

In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefLogit.COLO205",y="AltLogit.COLO205",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Logit")+
ylab("Alternate Allele Logit")+
ggtitle("COLO205"))


In [ ]:
plotnine.options.figure_size = (10,10)
(ggplot(snp_predictions,
        aes(x="RefCalibrated.COLO205",y="AltCalibrated.COLO205",label='Rsid',color='Target'))+
geom_point(alpha=1)+
theme_bw(20)+
xlab("Reference Allele Calibrated Predictions")+
ylab("Alternate Allele Calibrated Predictions")+
ggtitle("COLO205"))
